In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Step 1: 載入套件與模型（TensorFlow Hub）
#import tensorflow as tf
#import tensorflow_hub as hub
#import pandas as pd
#import numpy as np
#from PIL import Image
#from tqdm import tqdm
#from sklearn.metrics import accuracy_score

# CropNet 模型（Cassava Disease V1）
#model = hub.KerasLayer("https://tfhub.dev/google/cropnet/classifier/cassava_disease_V1/2")
# Step 2: 載入 cassava dataset 的 CSV 與圖檔路徑
#df = pd.read_csv("/kaggle/input/c/cassava-leaf-disease-classification/train.csv")
#df["image_path"] = df["image_id"].apply(lambda x: f"/kaggle/input/c/cassava-leaf-disease-classification/train_images/{x}")
# Step 3: 定義圖片預處理函數
#def preprocess_image(image_path):
#    image = Image.open(image_path).convert("RGB").resize((224, 224))
#    image = np.array(image) / 255.0
#    return np.expand_dims(image, axis=0)
# Step 4: 對所有圖片執行推論
#y_true = []
#y_pred = []

# 可選：只跑前 100 張做測試（速度快）
# df = df.head(100)

#for _, row in tqdm(df.iterrows(), total=len(df)):
#    img = preprocess_image(row["image_path"])
#    logits = model(img)
#    pred = tf.argmax(logits, axis=1).numpy()[0]
    
#    y_true.append(row["label"])
#    y_pred.append(pred)
# Step 5: 計算準確率
#acc = accuracy_score(y_true, y_pred)
#print(f"✅ Accuracy on Cassava training set: {acc:.2%}")
print(f"✅ Msg: All pass cell 1")

In [ ]:
import tensorflow as tf
from PIL import Image
import numpy as np
import os
from tqdm import tqdm
import pandas as pd

# Step 1: 載入模型（非 Keras 方式）
#model_path = "/kaggle/input/cropnetv1/tensorflow2/default/1"
#model = tf.saved_model.load(model_path)

# 印出 model 的可能 callable 區塊（通常有 __call__）
#print("模型是否可呼叫：", callable(model))
#print("model.__call__ exists:", hasattr(model, '__call__'))

# Step 2: 圖片預處理函數
#def preprocess_image(image_path):
#    image = Image.open(image_path).convert("RGB").resize((224, 224))
#    image = np.array(image) / 255.0
#    image = tf.convert_to_tensor(image, dtype=tf.float32)
#    return tf.expand_dims(image, axis=0)

# Step 3: 載入測試集
#submission_df = pd.read_csv("/kaggle/input/c/cassava-leaf-disease-classification/sample_submission.csv")
#test_dir = "/kaggle/input/c/cassava-leaf-disease-classification/test_images"

# Step 4: 推論（如果 model 是可呼叫的）
#predictions = []
#for image_id in tqdm(submission_df["image_id"]):
#    image_path = os.path.join(test_dir, image_id)
#    img = preprocess_image(image_path)
    ##logits = model(img)  # 若 model 不可呼叫這裡會噴錯
#    logits = model(img, training=False)
#    pred = tf.argmax(logits, axis=1).numpy()[0]
#    predictions.append(pred)

# Step 5: 儲存 CSV
#submission_df["label"] = predictions
#submission_df.to_csv("submission.csv", index=False)
#print("✅ submission.csv 已產生！")

In [ ]:
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image
import os
import pandas as pd
from tqdm import tqdm
import sys

# 確認壓縮檔展開後的結構
base_dir = "/kaggle/input/csft-cassava-model-v9"
for root, dirs, files in os.walk(base_dir):
    print("📁", root)
    for f in files:
        print("   └──", f)
        
#加上這段來 debug
print("✅ 目前所有 Kaggle Input 路徑：")
!ls /kaggle/input
!ls /kaggle/input/c
!ls /kaggle/input/c/cassava-leaf-disease-classification


# ✅ 加入正確的 models 模組路徑
sys.path.append('/kaggle/input/csft-cassava-model-v9/pytorch/default/1/models')
#sys.path.append('/kaggle/input/csft-cassava-model-v8/pytorch/default/1/csft-cassava-model/models')
# ✅ 匯入模型定義
from csft_module import CrossScaleFusionTransformer

# Step 1: 讀入模型
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CrossScaleFusionTransformer(img_size=512, num_classes=5)  # 根據Train設定
# ✅ 正確的模型參數檔路徑
#model.load_state_dict(torch.load("/kaggle/input/csft-cassava-model-v7/fold1.pt", map_location=device))  #  改成實際路徑
model.load_state_dict(torch.load("/kaggle/input/csft-cassava-model-v9/pytorch/default/1/fold1.pt", map_location=device))
model.to(device)
model.eval()

# Step 2: 定義 preprocess（需與訓練時一致）
transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

# Step 3: 讀入 sample_submission 和測試圖檔
#submission_df = pd.read_csv("/kaggle/input/cassava-leaf-disease-classification/sample_submission.csv")
#test_dir = "/kaggle/input/cassava-leaf-disease-classification/test_images"
submission_df = pd.read_csv("/kaggle/input/c/cassava-leaf-disease-classification/sample_submission.csv")
test_dir = "/kaggle/input/c/cassava-leaf-disease-classification/test_images"

# Step 4: 推論並寫入結果
predictions = []
for image_id in tqdm(submission_df["image_id"]):
    image_path = os.path.join(test_dir, image_id)
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)
    
    # CSFT 是三路輸入，給同樣圖像三次
    with torch.no_grad():
        output = model(image, image, image)
        pred = torch.argmax(output, dim=1).item()
        predictions.append(pred)

# Step 5: 輸出 CSV
submission_df["label"] = predictions
submission_df.to_csv("submission.csv", index=False)
print("✅ submission.csv 已產生")